In [ ]:
import pandas as pd
import geopandas as gp
import yaml
from pathlib import Path
import numpy as np

In [ ]:
base_run_path = Path("../pycap_runs/pycap_base")
econ_path = Path('../econ')

### first get the well IDs from the pycap yml configuration file

In [ ]:
with open(base_run_path / "LPR_Redux.yml", 'r') as ifp:
        indat = yaml.safe_load(ifp)

In [ ]:
wells = [int(i.split('_')[1]) for i in list(indat.keys()) if i.startswith('well')]

### we need to cross reference these IDs to align with the other datasets below. The IDs in the Yml file are hicap IDs - we need generic site numbers

In [ ]:
well_source_xref = pd.read_csv(econ_path / "source_hcw_xref.csv", index_col=1).dropna()
well_source_xref.index = [int(i) if not np.isnan(i) else 0 for i in well_source_xref.index]

In [ ]:
well_source_xref

In [ ]:
src_wells = [well_source_xref.loc[w, 'site_no'] 
             for w in wells
             if w in well_source_xref.index ]

In [ ]:
len(src_wells), len(wells)

In [ ]:
crop_rotations = gp.read_file('../econ/parcel_rot.shp').dropna()

In [ ]:
crop_rotations.explore(column='rotation')

In [ ]:
source2parcel = pd.read_csv('../econ/source_parcel_xref.csv')
source2parcel

In [ ]:
wells_w_acres = pd.DataFrame(index=src_wells)

In [ ]:
wells_w_acres = wells_w_acres.merge(source2parcel, left_index=True, right_on='site_no')
wells_w_acres = wells_w_acres.merge(crop_rotations[['parcel_id', 'rotation', 'ag_acres']],
                                    left_on='parcel_id', right_on='parcel_id')

In [ ]:
wells_w_acres.rotation.unique()

In [ ]:
wells_w_acres


### now we need to calculate the per-acre dollar value for each rotation

#### source for rotation percentages:
https://widnr.widen.net/view/pdf/z8j5lsfp00/DG_CSLSAppendixF_2021.pdf?t.download=true
https://p.widencdn.net/8ghipa/Wiscland_2_User_Guide_September_2016

#### source for per acre costs
http://nass.usda.gov/Statistics_by_State/Wisconsin/Publications/Annual_Statistical_Bulletin/2024AgStats_WI.pdf

In [ ]:
rotations = pd.read_excel(econ_path / 'CropValuation.xlsx', 
                          sheet_name='Rotations',
                          index_col=1).drop(columns="Crop")
rotations

In [ ]:
per_crop_values = pd.read_excel(econ_path / 'CropValuation.xlsx', 
                          sheet_name='Crop_Values',
                          usecols="F,G", index_col=0).dropna()
per_crop_values

In [ ]:
rotations = rotations.merge(per_crop_values, left_index=True, right_index=True)

In [ ]:
for cc in rotations:
    if 'dollar' not in cc:
        rotations[cc] *= rotations['dollars_per_acre']

In [ ]:
receipts = rotations[[cc for cc in rotations.columns
                   if 'dollar' not in cc]].sum()
receipts.loc['non-ag'] = 0

# now we pull it all together 

In [ ]:
wells_w_acres['receipts'] = [receipts.loc[i] for i in wells_w_acres.rotation]

In [ ]:
wells_w_acres['acres_irrigated'] = wells_w_acres.source_to_parcel_pct * wells_w_acres.ag_acres
wells_w_acres['marginal_ag_receipts'] = wells_w_acres['acres_irrigated'] * wells_w_acres.receipts

In [ ]:
wells_w_acres

In [ ]:
receipts_summary = wells_w_acres.groupby('site_no').sum().rename(
    columns={'marginal_ag_receipts':'total_receipts'}
)[['total_receipts','acres_irrigated']]

In [ ]:
(receipts_summary.total_receipts/receipts_summary.acres_irrigated).hist(bins=50)

In [ ]:
# flip the index back to hi-cap well numbers to align with the yml file
receipts_summary = receipts_summary.merge(well_source_xref,left_index=True,right_on='site_no')

In [ ]:
receipts_summary.to_csv(econ_path / 'total_receipts.csv')